In [1]:
# !pip install -q openfe

In [2]:
import sys
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
import xgboost as xgb
import lightgbm as lgbm
import statistics
from sklearn.linear_model import Ridge
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, SplineTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.base import clone
from functools import partial
from joblib import dump
from scipy.stats import skew, kurtosis
from openfe import OpenFE, transform
import warnings

warnings.filterwarnings('ignore')


/home/bk_anupam/anaconda3/envs/fastai/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/bk_anupam/anaconda3/envs/fastai/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [3]:
sys.path.append(os.path.abspath("/home/bk_anupam/code/ML/ML_UTILS/"))

In [4]:
import train_tabular_utils as tt
import cv_split_utils
import enums
import data_utils

In [5]:
class Config:
    RUN_MODE = "LOCAL"
    RANDOM_SEED = 1
    NUM_FOLDS = 5
    TARGET_COL_NAME = "FloodProbability"        
    SCALER = enums.Scaler.StandardScaler
    METRIC = enums.Metrics.R2
    # These values are more dynamic   
    MODEL_TYPE = enums.ModelName.Ridge    
    NUM_TUNING_TRIALS = 2
    TUNE_ON_SINGLE_FOLD = True
    TRAIN_SINGLE_FOLD = False
    GENERATE_AUTO_FEATURES = False
    PERSIST_MODEL = False
    TRANSFORM_TARGET = False

COLS_TO_LEAVE = ["FloodProbability", "kfold"]
CPU_COUNT = os.cpu_count()

DATA_READPATH = "./data/"
DATA_WRITEPATH = "./output/"
SUBMISSION_FILEPATH = DATA_READPATH
if Config.RUN_MODE == "KAGGLE":
    # If we are not generating features, we are using already generated features
    if Config.GENERATE_AUTO_FEATURES:
        DATA_READPATH = "/kaggle/input/playground-series-s4e5/"
        SUBMISSION_FILEPATH = DATA_READPATH
    else:
        DATA_READPATH = "/kaggle/input/playground-series-s4e5/"
        SUBMISSION_FILEPATH = "/kaggle/input/playground-series-s4e5/"
    DATA_WRITEPATH = "/kaggle/working/"

In [6]:
model_static_params = {
    enums.ModelName.XGBoost: {
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "seed": Config.RANDOM_SEED,
        "verbosity": 0,
    },
    enums.ModelName.LGBM: {
        "objective": "root_mean_squared_error",
        "metric": 'rmse',
        "verbosity": -1,    # <0: fatal, =0: error (warn), =1: info, >1: debug
        "boosting_type": "gbdt"
    },
    enums.ModelName.CatBoost: {
        "objective": "RMSE",
        "verbose": 0,
        "random_seed": Config.RANDOM_SEED,
        "eval_metric": "RMSE",
        'grow_policy':  'Lossguide',
        'bootstrap_type': 'Poisson',
        'task_type': 'GPU'
    },
    enums.ModelName.RandomForest: {
        "random_state": Config.RANDOM_SEED,
        "n_jobs": -1
    },
    enums.ModelName.Ridge: {
        "random_state": Config.RANDOM_SEED
    }
}

In [7]:
tuned_model_params = None

In [8]:
# import train dataset locally from data folder
df_train = pd.read_csv(DATA_READPATH + "train.csv", index_col='id')
# import test dataset locally from data folder
df_test = pd.read_csv(DATA_READPATH + "test.csv", index_col='id')
# keep a copy of original train and test data for later use
# df_train_orig = df_train.copy()
# df_test_orig = df_test.copy()

In [9]:
feature_cols_for_fe = df_test.columns.to_list()

In [10]:
# Function to compute skewness and kurtosis for each row
def compute_skew_kurtosis(matrix):
    skewness = skew(matrix, axis=1)
    kurt = kurtosis(matrix, axis=1)
    return skewness, kurt

def create_features(df, feature_cols):
    # Create a new feature by summing all features
    df["f_sum"] = df[feature_cols].sum(axis=1)
    # Create a new feature by taking mean of all features
    df["f_mean"] = df[feature_cols].mean(axis=1)
    df["f_median"] = df[feature_cols].median(axis=1)
    # standard deviation
    df['f_std'] = df[feature_cols].std(axis=1)
    # min and max
    df['f_min'] = df[feature_cols].min(axis=1)
    df['f_max'] = df[feature_cols].max(axis=1)
    # Compute skewness and kurtosis
    skewness, kurt = compute_skew_kurtosis(df[feature_cols].values)
    df['f_skew'] = skewness
    df['f_kurtosis'] = kurt    
    # Quantiles
    quantiles = [0.25, 0.5, 0.75]
    for q in quantiles:
        df[f'f_quantile_{int(q*100)}'] = df[feature_cols].quantile(q=q, axis=1)        
    # sorted features
    sorted_features = [f"sort_{i}" for i in np.arange(len(feature_cols))]
    df[sorted_features] = np.sort(df[feature_cols], axis=1)
    return df

In [11]:
df_train = create_features(df_train, feature_cols_for_fe)
df_test = create_features(df_test, feature_cols_for_fe)

In [12]:
df_train.head()

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,f_mean,f_median,f_std,f_min,f_max,f_skew,f_kurtosis,f_quantile_25,f_quantile_50,f_quantile_75
id,,,,,,,,,,,,,,,,,,,,,
0,5,8,5,8,6,4,4,3,3,4,...,4.70,4.5,1.750188,2,8,0.533028,-0.685939,3.00,4.5,5.25
1,6,7,4,4,8,8,3,5,4,6,...,4.70,4.0,2.296450,0,9,0.136973,-0.560579,3.00,4.0,6.25
2,6,5,6,7,3,7,1,5,4,5,...,4.95,5.0,1.932411,1,8,-0.376816,-0.855085,3.00,5.0,6.25
3,3,4,6,5,4,8,4,7,6,8,...,5.20,5.0,1.641565,2,8,0.111328,-0.738770,4.00,5.0,6.25
4,5,3,2,6,4,4,3,3,3,3,...,3.60,3.0,1.500877,1,6,0.233825,-0.993012,2.75,3.0,5.00


In [13]:
feature_cols= [x for x in df_train.columns.to_list() if x not in COLS_TO_LEAVE]
print(f"len(feature_cols)={len(feature_cols)}")

len(feature_cols)=31


In [14]:
scaler = StandardScaler()
onehot_encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
polynomial_features = PolynomialFeatures(degree=2, include_bias=False)
spline_transformer = SplineTransformer(n_knots=5, degree=3, include_bias=False)
preprocessor = ColumnTransformer(
    transformers=[        
        #("poly", polynomial_features, feature_cols),
        ("scaler", scaler, feature_cols),
        ("onehot", onehot_encoder, ['f_sum']),        
    ], remainder="passthrough"
)

In [15]:
def train_and_validate(model_name, model, df, feature_cols, target_col_name, metric, n_repeat=1, single_fold=False, num_folds=5):    
    df = cv_split_utils.kfold_dataframe(df, random_state=Config.RANDOM_SEED, num_folds=Config.NUM_FOLDS)
    df_oof_preds = pd.DataFrame()
    fold_metrics_model = []
    for fold in range(num_folds):
        fold_model = clone(model)
        df_train_fold, df_val_fold = tt.get_fold_df(df, fold)
        train_X, train_y, val_X, val_y = tt.get_train_val_nparray(df_train_fold, df_val_fold, feature_cols, target_col_name)
        fold_model.fit(train_X, train_y)
        val_y_pred = fold_model.predict(val_X)
        fold_val_metric = tt.get_eval_metric(metric, val_y, val_y_pred)
        print(f"Fold {fold} - {model_name} - {metric} : {fold_val_metric}")
        df_fold_val_preds = df_val_fold[['kfold', target_col_name]]
        df_fold_val_preds['oof_preds'] = val_y_pred
        df_oof_preds = pd.concat([df_oof_preds, df_fold_val_preds], axis=0)
        fold_metrics_model.append((fold_val_metric, fold_model))
        if single_fold:
            break
    cv = tt.get_eval_metric(metric, df_oof_preds[target_col_name], df_oof_preds['oof_preds'] )
    print(f"{model_name} metric={metric} CV score = {cv}")
    metrics = [item[0] for item in fold_metrics_model]
    mean_metric, std_metric = tt.get_metric_stats(metrics)    
    print(f"{model_name} Mean {metric} = {mean_metric}, std = {std_metric}")        
    return fold_metrics_model, df_oof_preds

In [ ]:
def get_lgbm_tuning_params(trial):    
    params_dynamic = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=25),
        'max_depth': trial.suggest_int('max_depth', 4, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
        'num_leaves': trial.suggest_int('num_leaves', 4, 256, step=4),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 100),
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 50, 500, step=20)
    }
    return {**model_static_params[enums.ModelName.LGBM], **params_dynamic}

In [16]:
def get_catboost_tuning_params(trial):
    params_dynamic = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 5000, step=50),
        'max_depth': trial.suggest_int('max_depth', 4, 16),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1),
        # comment colsample_bylevel for GPU training
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1),
        'num_leaves': trial.suggest_int('num_leaves', 4, 256, step=4),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 100),
        'random_strength': trial.suggest_loguniform('random_strength', 0.01, 10),
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 50, 500, step=20),
        'max_bin': trial.suggest_int('max_bin', 32, 255)
    }
    return {**model_static_params[enums.ModelName.CatBoost], **params_dynamic}

In [ ]:
def get_xgb_tuning_params(trial):
    params_dynamic = {            
            'n_estimators': trial.suggest_int('n_estimators', 100, 5000, step=100),
            'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1.0, log=True),
            'max_depth': trial.suggest_int('max_depth', 4, 32),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'gamma': trial.suggest_float('gamma', 0, 1),
            'subsample': trial.suggest_float('subsample', 0.5, 1),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
            'reg_lambda': trial.suggest_float('reg_lambda', 1, 100),
            'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 100, 500, step=20)
        }
    return {**model_static_params[enums.ModelName.XGBoost], **params_dynamic}

In [17]:
def get_model_tuning_params(trial, model_name):
    if model_name == enums.ModelName.Ridge:
        return {
            "alpha": trial.suggest_float("alpha", 1e-4, 1e4, log=True)
        }
    if model_name == enums.ModelName.Lasso:
        return {
            "alpha": trial.suggest_float("alpha", 1e-4, 1e4, log=True)
        }
    if model_name == enums.ModelName.RandomForest:
        return {        
            "n_estimators": trial.suggest_int("n_estimators", 400, 3000, step=100),
            "max_depth": trial.suggest_int("max_depth", 10, 30),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 2, 16),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 16),
            "max_features": trial.suggest_categorical("max_features", ["log2", "sqrt", None])
        }
    if model_name == enums.ModelName.CatBoost:
        return get_catboost_tuning_params(trial)
    if model_name == enums.ModelName.LGBM:
        return get_lgbm_tuning_params(trial)
    if model_name == enums.ModelName.XGBoost:
        return get_xgb_tuning_params(trial)

In [18]:
def hyperparams_tuning_objective(trial, model_name, preprocessor, df_train,  
                                 feature_cols, metric, target_col_name, single_fold=False, num_folds=5):               
    model_params = get_model_tuning_params(trial, model_name)
    model = tt.get_model(model_name=model_name, params=model_params, metric=metric)
    if preprocessor is not None:
        model = make_pipeline(preprocessor, model)
    fold_metrics_model, df_val_preds = train_and_validate(
                                        model_name=model_name,
                                        model=model,
                                        df=df_train,
                                        feature_cols=feature_cols,
                                        target_col_name=target_col_name,
                                        metric=metric,
                                        single_fold=single_fold,
                                        num_folds=num_folds
                                    )
    fold_metrics = [x[0] for x in fold_metrics_model]
    mean_metric = statistics.mean(fold_metrics)                
    return mean_metric

In [19]:
def tune_model_params(study_name, study_direction, num_trials, model_name, preprocessor,
                      df_train,  feature_cols, metric, target_col_name, 
                      single_fold=False, num_folds=5):
    model_params_tuning_obj_partial = partial(
        hyperparams_tuning_objective,
        model_name=model_name,
        preprocessor=preprocessor,        
        df_train=df_train,
        feature_cols=feature_cols,
        metric=metric,
        target_col_name=target_col_name,
        single_fold=single_fold,
        num_folds=num_folds
    )
    study = optuna.create_study(direction=study_direction, study_name=study_name)
    study.optimize(model_params_tuning_obj_partial, n_trials=num_trials)
    best_trial = study.best_trial
    print(f"Best trial: number = {best_trial.number}, value = {best_trial.value}, params = {best_trial.params}")
    return best_trial.params

In [20]:
if tuned_model_params is None:
    tuned_model_params = tune_model_params(
                            study_name=Config.MODEL_TYPE + "_ModelTuning", 
                            study_direction="maximize",
                            num_trials=Config.NUM_TUNING_TRIALS,
                            model_name=Config.MODEL_TYPE,
                            preprocessor=preprocessor,
                            df_train=df_train.sample(frac=0.1),
                            feature_cols=feature_cols,
                            metric=Config.METRIC,
                            target_col_name=Config.TARGET_COL_NAME,
                            single_fold=Config.TUNE_ON_SINGLE_FOLD,
                            num_folds=Config.NUM_FOLDS
                    )

[I 2024-05-31 19:11:17,292] A new study created in memory with name: Ridge_ModelTuning
[I 2024-05-31 19:11:24,110] Trial 0 finished with value: 0.865830794470718 and parameters: {'alpha': 11.305059973897613}. Best is trial 0 with value: 0.865830794470718.


Fold 0 - Ridge - R2 : 0.865830794470718
Ridge metric=R2 CV score = 0.865830794470718
Ridge Mean R2 = 0.865830794470718, std = 0.0


[I 2024-05-31 19:11:34,504] Trial 1 finished with value: 0.8644737715286331 and parameters: {'alpha': 1740.0928698270993}. Best is trial 0 with value: 0.865830794470718.


Fold 0 - Ridge - R2 : 0.8644737715286331
Ridge metric=R2 CV score = 0.8644737715286331
Ridge Mean R2 = 0.8644737715286331, std = 0.0
Best trial: number = 0, value = 0.865830794470718, params = {'alpha': 11.305059973897613}


In [21]:
model_params = None
params_static = model_static_params.get(Config.MODEL_TYPE)
if params_static is not None and tuned_model_params is not None:
    model_params = {**model_static_params[Config.MODEL_TYPE], **tuned_model_params}
else:
    model_params = tuned_model_params

In [22]:
# model_params = model_static_params.get(Config.MODEL_TYPE)

In [23]:
model = tt.get_model(model_name=Config.MODEL_TYPE, params=model_params, metric=Config.METRIC)
model_pipeline = make_pipeline(preprocessor, model)

In [24]:
def persist(model_name, fold_metrics_model, df_oof_preds, persist_model=False, output_path=""):    
    fold_models = [item[1] for item in fold_metrics_model]    
    if persist_model:
        for index, model in enumerate(fold_models):
            fold_model_name = output_path + f"{model_name}_{index}.joblib"        
            dump(model, fold_model_name)
            print(f"saved {fold_model_name}")    
    df_oof_preds.to_csv(output_path + f"df_val_preds_{model_name}.csv")
    print(f"Saved validation data predictions to df_val_preds_{model_name}.csv")  

In [25]:
fold_metrics_model, df_oof_preds = train_and_validate(
                                        model_name=Config.MODEL_TYPE,
                                        model=model_pipeline,
                                        df=df_train,
                                        feature_cols=feature_cols,
                                        target_col_name=Config.TARGET_COL_NAME,
                                        metric=Config.METRIC,
                                        single_fold=Config.TRAIN_SINGLE_FOLD,
                                        num_folds=Config.NUM_FOLDS
                                    )

persist(
        model_name=Config.MODEL_TYPE, 
        fold_metrics_model=fold_metrics_model, 
        df_oof_preds=df_oof_preds, 
        persist_model=Config.PERSIST_MODEL, 
        output_path=DATA_WRITEPATH
)

Fold 0 - Ridge - R2 : 0.865830794470718
Fold 1 - Ridge - R2 : 0.866118455601935
Fold 2 - Ridge - R2 : 0.8656391248908174
Fold 3 - Ridge - R2 : 0.8658288329839805
Fold 4 - Ridge - R2 : 0.8661879207998384
Ridge metric=R2 CV score = 0.865921317765004
Ridge Mean R2 = 0.8659210257494578, std = 0.00020313581581711044
Saved validation data predictions to df_val_preds_Ridge.csv


In [26]:
# For each fold, get the test predictions using corresponding fold model
df_fold_test_preds = tt.get_fold_test_preds(
                        fold_metrics_model,
                        df_test = df_test,
                        feature_cols = feature_cols,
                        num_folds = Config.NUM_FOLDS,
                    )
fold_metrics = [item[0] for item in fold_metrics_model]
# Since for RMSLE metric lower is better, we take the inverse of fold metric value to get its weight    
fold_weights = 1 / np.array(fold_metrics)
# normalize the fold weights
fold_weights = fold_weights / np.sum(fold_weights)
# Combine fold predictions using simple averaging    
df_fold_test_preds["test_preds"] = tt.combine_fold_test_preds(df_fold_test_preds, fold_weights=None)
print(f"Completed prediction for {len(df_test)} test rows")

Completed prediction for 745305 test rows


In [27]:
df_submission = pd.read_csv(SUBMISSION_FILEPATH + 'sample_submission.csv')
df_submission[Config.TARGET_COL_NAME]= df_fold_test_preds["test_preds"]
df_submission.to_csv(DATA_WRITEPATH + f'submission_{Config.MODEL_TYPE}.csv',index=False)
df_fold_test_preds.to_csv(DATA_WRITEPATH + f'{Config.MODEL_TYPE}_test_preds.csv',index=False)
df_submission.head()

,id,FloodProbability
0,1117957,0.577364
1,1117958,0.452056
2,1117959,0.451743
3,1117960,0.472565
4,1117961,0.472270


In [28]:
# import datetime
# from sklearn.model_selection import KFold
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import r2_score
# from colorama import Fore, Style


# kf = KFold(n_splits=5, shuffle=True, random_state=1)

# SINGLE_FOLD = True

# def cross_validate(model, train, label, features=feature_cols, n_repeats=1):
#     """Compute out-of-fold and test predictions for a given model.
    
#     Out-of-fold and test predictions are stored in the global variables
#     oof and test_pred, respectively.
    
#     If n_repeats > 1, the model is trained several times with different seeds.
#     """
#     start_time = datetime.datetime.now()
#     scores = []
#     oof_preds = np.full_like(train.FloodProbability, np.nan, dtype=float)
#     for fold, (idx_tr, idx_va) in enumerate(kf.split(train)):
#         X_tr = train.iloc[idx_tr][features]
#         X_va = train.iloc[idx_va][features]
#         y_tr = train.iloc[idx_tr].FloodProbability
#         y_va = train.iloc[idx_va].FloodProbability
        
#         y_pred = np.zeros_like(y_va, dtype=float)
#         for i in range(n_repeats):
#             m = clone(model)
#             if n_repeats > 1:
#                 mm = m
#                 if isinstance(mm, Pipeline):
#                     mm = mm[-1]
#                 mm.set_params(random_state=i)
#             m.fit(X_tr, y_tr)
#             y_pred += m.predict(X_va)
#         y_pred /= n_repeats                
#         score = r2_score(y_va, y_pred)
#         print(f"# Fold {fold}: R2={score:.5f}")
#         scores.append(score)
#         oof_preds[idx_va] = y_pred
#         if Config.TRAIN_SINGLE_FOLD: break
            
#     elapsed_time = datetime.datetime.now() - start_time
#     print(f"{Fore.GREEN}# Overall: {np.array(scores).mean():.5f} {label}"
#           f"{' single fold' if SINGLE_FOLD else ''}"
#           f"   {int(np.round(elapsed_time.total_seconds() / 60))} min{Style.RESET_ALL}")

In [29]:
#cross_validate(model_pipeline, df_train, "Ridge", features=feature_cols)

In [30]:
# from sklearn.metrics import r2_score

# r2_score(df_train.FloodProbability, (df_train[feature_cols_for_fe].sum(axis=1) * 0.0056) - 0.05)